In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd


def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(slp_time)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [2]:
#This line will open a new chrome window and start the scraping.

path ='/Users/prahlad/Downloads/chromedriver'


df = get_jobs("data analyst", 400, False,path,10)

 x out worked
Progress: 0/400
Progress: 1/400
Progress: 2/400
Progress: 3/400
Progress: 4/400
Progress: 5/400
Progress: 6/400
Progress: 7/400
Progress: 8/400
Progress: 9/400
Progress: 10/400
Progress: 11/400
Progress: 12/400
Progress: 13/400
Progress: 14/400
Progress: 15/400
Progress: 16/400
Progress: 17/400
Progress: 18/400
Progress: 19/400
Progress: 20/400
Progress: 21/400
Progress: 22/400
Progress: 23/400
Progress: 24/400
Progress: 25/400
Progress: 26/400
Progress: 27/400
Progress: 28/400
Progress: 29/400
 x out failed
Progress: 30/400
Progress: 31/400
Progress: 32/400
Progress: 33/400
Progress: 34/400
Progress: 35/400
Progress: 36/400
Progress: 37/400
Progress: 38/400
Progress: 39/400
Progress: 40/400
Progress: 41/400
Progress: 42/400
Progress: 43/400
Progress: 44/400
Progress: 45/400
Progress: 46/400
Progress: 47/400
Progress: 48/400
Progress: 49/400
Progress: 50/400
Progress: 51/400
Progress: 52/400
Progress: 53/400
Progress: 54/400
Progress: 55/400
Progress: 56/400
Progress: 57/

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Job Title          400 non-null    object
 1   Salary Estimate    400 non-null    object
 2   Job Description    400 non-null    object
 3   Rating             400 non-null    object
 4   Company Name       400 non-null    object
 5   Location           400 non-null    object
 6   Headquarters       400 non-null    object
 7   Size               400 non-null    object
 8   Founded            400 non-null    object
 9   Type of ownership  400 non-null    object
 10  Industry           400 non-null    object
 11  Sector             400 non-null    object
 12  Revenue            400 non-null    object
 13  Competitors        400 non-null    object
dtypes: object(14)
memory usage: 43.9+ KB


In [4]:
# write to csv 
df.to_csv('gd_scraper_da.csv')

In [5]:
df1=get_jobs("data scientist", 400, False,path,10)

 x out worked
Progress: 0/400
Progress: 1/400
Progress: 2/400
Progress: 3/400
Progress: 4/400
Progress: 5/400
Progress: 6/400
Progress: 7/400
Progress: 8/400
Progress: 9/400
Progress: 10/400
Progress: 11/400
Progress: 12/400
Progress: 13/400
Progress: 14/400
Progress: 15/400
Progress: 16/400
Progress: 17/400
Progress: 18/400
Progress: 19/400
Progress: 20/400
Progress: 21/400
Progress: 22/400
Progress: 23/400
Progress: 24/400
Progress: 25/400
Progress: 26/400
Progress: 27/400
Progress: 28/400
Progress: 29/400
 x out failed
Progress: 30/400
Progress: 31/400
Progress: 32/400
Progress: 33/400
Progress: 34/400
Progress: 35/400
Progress: 36/400
Progress: 37/400
Progress: 38/400
Progress: 39/400
Progress: 40/400
Progress: 41/400
Progress: 42/400
Progress: 43/400
Progress: 44/400
Progress: 45/400
Progress: 46/400
Progress: 47/400
Progress: 48/400
Progress: 49/400
Progress: 50/400
Progress: 51/400
Progress: 52/400
Progress: 53/400
Progress: 54/400
Progress: 55/400
Progress: 56/400
Progress: 57/

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Job Title          400 non-null    object
 1   Salary Estimate    400 non-null    object
 2   Job Description    400 non-null    object
 3   Rating             400 non-null    object
 4   Company Name       400 non-null    object
 5   Location           400 non-null    object
 6   Headquarters       400 non-null    object
 7   Size               400 non-null    object
 8   Founded            400 non-null    object
 9   Type of ownership  400 non-null    object
 10  Industry           400 non-null    object
 11  Sector             400 non-null    object
 12  Revenue            400 non-null    object
 13  Competitors        400 non-null    object
dtypes: object(14)
memory usage: 43.9+ KB


In [9]:
df1.to_csv('gd_scraper_ds.csv')

In [10]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Marketing Data Analyst,$35K-$65K (Glassdoor est.),Overview\n\n\nAs a key member of the Analytics...,3.5,Efinancial\n3.5,"Tempe, AZ","Bellevue, WA",201 to 500 employees,2001,Company - Private,Insurance Agencies & Brokerages,Insurance,$50 to $100 million (USD),-1
1,Data Analyst,$35K-$65K (Glassdoor est.),What are we looking for in a Data Analyst?\n\n...,2.6,Comprehensive Healthcare\n2.6,"Yakima, WA","Yakima, WA",501 to 1000 employees,1971,Nonprofit Organization,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1
2,Data Analyst/Programmer,$35K-$65K (Glassdoor est.),Data Analyst/Programmer\n\nMarks Paneth LLP is...,3.8,Mark Paneth\n3.8,"Washington, DC","New York, NY",501 to 1000 employees,1907,Private Practice / Firm,Accounting,Accounting & Legal,$100 to $500 million (USD),-1
3,Data Analyst,$35K-$65K (Glassdoor est.),WHO WE ARE\n\nNCSOFT is a premiere digital ent...,3.0,NCSOFT\n3.0,"San Mateo, CA","Seoul, South Korea",1001 to 5000 employees,1997,Company - Public,Video Games,Media,$10+ billion (USD),"Blizzard Entertainment, Riot Games, Electronic..."
4,Data Analyst,$35K-$65K (Glassdoor est.),The mission of Success Academies is to reimagi...,3.1,Success Academy Charter Schools\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,2006,School / School District,K-12 Education,Education,$100 to $500 million (USD),-1
